In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv('order_brush_order.csv')
df.head()

,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [14]:
df["userid"].value_counts()

124597967    97
50167978     63
78206381     49
145795810    47
95399773     46
             ..
149052813     1
141972876     1
42031659      1
33577101      1
10245         1
Name: userid, Length: 93883, dtype: int64

In [15]:
# Create a new column `unix_time` by converting `event_time` to the corresponding Unix time.
time_zero = pd.to_datetime('1970-01-01')
df["unix_time"] = pd.to_datetime(df["event_time"]).apply(lambda x: int((x - time_zero).total_seconds()))

# From here onwards, we actually only need `unit_time`, `shopid`, `userid` columns. But I'm too lazy to remove the other columns.

# Sort the rows of `df` ascendingly by `unit_time`. 
df = df.sort_values('unix_time')

In [16]:
df

,orderid,shopid,userid,event_time,unix_time
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,1577404800
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,1577404800
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,1577404800
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,1577404802
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,1577404805
...,...,...,...,...,...
114113,31507183252446,149254894,193333760,2019-12-31 23:59:43,1577836783
206585,31507187390691,147941492,40258063,2019-12-31 23:59:47,1577836787
126212,31507191066628,187123853,2338306,2019-12-31 23:59:51,1577836791
160657,31507191066627,154074176,2338306,2019-12-31 23:59:51,1577836791


In [18]:
# Then, it formats the buyerid(s) and store this string as a value in shopid_to_userids dictionary, with the shopid as the key.

def get_suspicious_buyers(df):
    # df is a dataframe that contains all the orders for a unique shopid, sorted ascendingly by `unix_time`.
    assert df['shopid'].nunique() == 1
    
    # Reset the index of df so that we can reference each order by the dataframe index later.
    df = df.reset_index(drop=True)
    # Stores the no. of orders of df.
    N = len(df)
    # Create new column `suspicious_buyer` to record the orders that occurred in a brushing period.
    df['suspicious_buyer'] = False
    # Stores the current shopid.
    shopid = df['shopid'].iloc[0]
    
    # st_idx denotes the index of the first order in the 1hr time interval.
    # en_idx denotes the index of the last order in the 1hr time interval.
    for st_idx in range(N):
        for en_idx in range(st_idx, N):
            # If time difference between the current last order and the first order is more than 3600s (1hr),
            # we do not need to consider the next order as a possible last order because the orders are sorted
            # ascendingly by `unix_time`.
            if df.loc[en_idx, 'unix_time'] - df.loc[st_idx, 'unix_time'] > 3600:
                break
                
            # The first condition checks for all the possible cases whereby the 1hr time interval could occur at. 
            # For st_idx == 0, we are considering the cases whereby the start of the 1hr time interval occurs at or before the first order in the dataset. 
            # For en_idx == 0, we are considering the cases whereby the end of the 1hr time interval occurs at or after the last order in the dataset.
            # For df.loc[en_idx+1, 'unix_time'] - df.loc[st_idx-1, 'unix_time'] > 3600,
            # we are considering the cases whereby the start and end of the 1hr time interval occurs between two orders in the dataset, 
            # more specifically, the start of the 1hr interval occurs within interval (df.loc[st_idx-1, 'unix_time'], df.loc[st_idx, 'unix_time']]
            # while the end of the 1hr interval occurs within interval [df.loc[en_idx, 'unix_time'], df.loc[en_idx+1, 'unix_time']).
    
            # The second condition checks that the 1hr time interval has concentration >= 3.
            if (st_idx == 0 or en_idx == N - 1 or df.loc[en_idx+1, 'unix_time'] - df.loc[st_idx-1, 'unix_time'] > 3600) and (en_idx-st_idx+1 >= 3 * df.loc[st_idx:en_idx, 'userid'].nunique()):
                df.loc[st_idx:en_idx, 'suspicious_buyer'] = True
    
    # Get all orders from suspicious buyers.
    df = df[df['suspicious_buyer']]                               
    
    # Dictionary that maps userid to the total no. of orders for that userid during brushing periods.
    # We do not need to calculate the order proportion for each userid since that is proportionate to the total no. of orders.
    occur_dict = df['userid'].value_counts().to_dict()
    
    # Stores formatted output.
    output = ''
    # Stores maximum no. of orders amoung all userids.
    max_count = 0
    # Stores the userids which have the maximum no. of orders.
    userid_list = []
    
    # Creates the formatted output and stores it in shopid_to_userids dictionary.
    for k, v in sorted(occur_dict.items()):
        if v > max_count:
            max_count = v
            userid_list = [k]
        elif v == max_count:
            userid_list.append(k) 
    for userid in userid_list:
        if output == '':
            output = str(userid)
        else:
            output += ('&' + str(userid))
            
    shopid_to_userids[shopid] = output

In [19]:
# Use groupby on `shopid` and call get_suspicious_buyers for each shopid.

# Running this cell takes about 1hr to complete.
shopid_to_userids = {}
df.groupby('shopid', as_index=False).apply(get_suspicious_buyers)

""


In [20]:
output_df = pd.DataFrame({'shopid': df['shopid'].unique()})
output_df['userid'] = '0'

# Transfer values from shopid_to_userids to output dataframe.
for i, row in output_df.iterrows():
    shopid = row['shopid']
    if shopid_to_userids[shopid] != '':
        
        output_df.loc[i, 'userid'] = shopid_to_userids[shopid]
        
# Store output in CSV file.    
output_df.to_csv('submission.csv', index=False)

In [21]:
df_out = pd.read_csv('submission.csv')

In [22]:
df_out

,shopid,userid
0,6042309,0
1,104804492,0
2,8715449,9753706
3,190969466,0
4,2859407,0
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


In [24]:
df_out.dtypes

shopid     int64
userid    object
dtype: object

In [25]:
df

,orderid,shopid,userid,event_time,unix_time
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,1577404800
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,1577404800
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,1577404800
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,1577404802
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,1577404805
...,...,...,...,...,...
114113,31507183252446,149254894,193333760,2019-12-31 23:59:43,1577836783
206585,31507187390691,147941492,40258063,2019-12-31 23:59:47,1577836787
126212,31507191066628,187123853,2338306,2019-12-31 23:59:51,1577836791
160657,31507191066627,154074176,2338306,2019-12-31 23:59:51,1577836791


In [27]:
df.drop(columns = ["orderid"])

,shopid,userid,event_time,unix_time
150060,6042309,97707522,2019-12-27 00:00:00,1577404800
25985,104804492,97707522,2019-12-27 00:00:00,1577404800
81020,8715449,97707522,2019-12-27 00:00:00,1577404800
146598,190969466,170182475,2019-12-27 00:00:02,1577404802
80355,2859407,12532131,2019-12-27 00:00:05,1577404805
...,...,...,...,...
114113,149254894,193333760,2019-12-31 23:59:43,1577836783
206585,147941492,40258063,2019-12-31 23:59:47,1577836787
126212,187123853,2338306,2019-12-31 23:59:51,1577836791
160657,154074176,2338306,2019-12-31 23:59:51,1577836791


In [30]:
shopid = pd.DataFrame(df['shopid'])
shopid.describe()

,shopid
count,2.227500e+05
mean,9.433117e+07
std,5.695790e+07
min,1.000900e+04
25%,4.980267e+07
50%,9.033636e+07
75%,1.475053e+08
max,2.154352e+08


In [32]:
df_int = pd.DataFrame(df.select_dtypes(include=['int64']))
df_int

,orderid,shopid,userid,unix_time
150060,31075200506751,6042309,97707522,1577404800
25985,31075200506752,104804492,97707522,1577404800
81020,31075200506753,8715449,97707522,1577404800
146598,31075201870570,190969466,170182475,1577404802
80355,31075205798264,2859407,12532131,1577404805
...,...,...,...,...
114113,31507183252446,149254894,193333760,1577836783
206585,31507187390691,147941492,40258063,1577836787
126212,31507191066628,187123853,2338306,1577836791
160657,31507191066627,154074176,2338306,1577836791
